In [62]:
# looking for a specific field_name
def match_insert(ast, field_name, result):
    # base
    if field_name in result:
        return
    
    if type(ast) == dict:
        if field_name in ast:
#             print(ast[field_name])
            result[field_name] = ast[field_name]
            return
        else:
            for key in ast.keys():
                match_insert(ast[key], field_name, result)
    elif type(ast) == list:
        for item in ast:
            match_insert(item, field_name, result)

# looking for a specific field_name whose value is field_value
def dfs_insert(ast, field_name, field_value, result, rst_field_name):
    # base
    if rst_field_name in result:
        return

    if type(ast) == dict:
        if field_name in ast and field_value in ast[field_name]:
            result[rst_field_name] = ast
            return
        else:
            for key in ast.keys():
                dfs_insert(ast[key], field_name, field_value, result, rst_field_name)
    elif type(ast) == list:
        for item in ast:
            dfs_insert(item, field_name, field_value, result, rst_field_name)



In [98]:

import json
# Open JSON ast
with open('./source/jsonAst/plsql_insert_2.json') as f:
    ast = json.load(f)

# match insert
ast_insert = {}
match_insert(ast, 'insert_statement', ast_insert)
# print(ast_insert)

# match target
ast_target = {}
match_insert(ast_insert, 'insert_into_clause', ast_target)

# match source
ast_source = {}
dfs_insert(ast_insert, 'rule-path', 'select_statement', ast_source, 'source')


In [102]:
def extract_target(ast):
    # get table name
    lineage_target_tbl = {}
    tbl_field_val = 'general_table_ref'
    dfs_insert(ast, 'rule-path', tbl_field_val, lineage_target_tbl, 'source')
    tgt_tbl = lineage_target_tbl['source']['regular_id'][0]['value']

    # get columns
    raw_target_col = {}
    col_field_name = 'column_list'
    match_insert(ast, col_field_name, raw_target_col)

    lineage_target_col = []
    if col_field_name in raw_target_col:
        get_list_dft(raw_target_col[col_field_name], lineage_target_col)

    return tgt_tbl, lineage_target_col
    
    
extract_target(ast_target)

('tt_bnk_dependent_defaulting', [])

In [101]:
def extract_source(ast):
    # get source table
    raw_source_tbl = {}
    match_insert(ast, 'from_clause', raw_source_tbl)
#     print(lineage_target_tbl)
    src_tbl = get_source_tbl(raw_source_tbl['from_clause'])
    print(src_tbl)
    
    # get source columns
    raw_source_col = {}
    dfs_insert(ast, 'rule-path', 'selected_element', raw_source_col, 'source')
    src_col = get_source_col(raw_source_col['source'])
    print(src_col)
    return src_tbl, src_col
    
def get_source_tbl(ast):
    src_tbl = []
    if type(ast) == list:
        for item in ast:
            # multi tables
            if 'table_ref_list' in item:
                for tbl in item['table_ref_list']:

                    src_tbl.append(tbl['table_ref_aux'][0]['regular_id'][0]['value'])
            # single table
            elif 'rule-path' in item and 'table_ref_list' in item['rule-path']:
                src_tbl.append(item['regular_id'][0]['value'])
    return src_tbl

def get_source_col(ast):
    src_col = []
    if 'expressions' in ast and type(ast['expressions']) == list:
        for item in ast['expressions']:
            if 'object-type' in item and 'column_name' == item['object-type']:
                src_col.append(item['regular_id'][0]['value'])
    return src_col
    
    
extract_source(ast_source)

{'rule-path': 'table_ref', 'table_ref_aux': [{'rule-path': 'table_ref_aux_internal dml_table_expression_clause tableview_name identifier id_expression', 'object-type': 'tableview_name', 'regular_id': [{'class': 'token', 'type': 'REGULAR_ID', 'value': 'current_dependent_defaulting'}]}, {'whitespace': '_'}, {'rule-path': 'table_alias identifier id_expression', 'object-type': 'table_alias', 'regular_id': [{'class': 'token', 'type': 'REGULAR_ID', 'value': 'dd'}]}]}
{'whitespace': 'n'}


KeyError: 'table_ref_aux'